In [1]:
# !pip install bs4

In [2]:
# !pip install beautifulsoup4

In [3]:
# !pip install pandas

In [4]:
# !pip install requests

In [5]:
import pandas as pd
import requests

In [6]:
from bs4 import BeautifulSoup

In [ ]:
# GitHub API endpoint for searching users
url = "https://api.github.com/search/users"
headers = {'Authorization': 'token '}

# Parameters for the API request
params = {
    'q': 'followers:>500 location:london',  # Search query
    'type': 'Users',
    'per_page': 100,  # Maximum allowed per page is 100
    'page': 1         # Start at the first page
}

all_users = []
while True:
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    users = data.get('items', [])
    all_users.extend(users)  # Add users from the current page

    # Check if there's another page
    if len(users) < params['per_page']:  # Less than `per_page` means we're at the last page
        break

    params['page'] += 1  # Go to the next page

In [8]:
len(all_users)

325

In [9]:
user0=requests.get(all_users[0]["url"]).json() 

In [10]:
user0

{'login': 'tj',
 'id': 25254,
 'node_id': 'MDQ6VXNlcjI1MjU0',
 'avatar_url': 'https://avatars.githubusercontent.com/u/25254?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/tj',
 'html_url': 'https://github.com/tj',
 'followers_url': 'https://api.github.com/users/tj/followers',
 'following_url': 'https://api.github.com/users/tj/following{/other_user}',
 'gists_url': 'https://api.github.com/users/tj/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/tj/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/tj/subscriptions',
 'organizations_url': 'https://api.github.com/users/tj/orgs',
 'repos_url': 'https://api.github.com/users/tj/repos',
 'events_url': 'https://api.github.com/users/tj/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/tj/received_events',
 'type': 'User',
 'user_view_type': 'public',
 'site_admin': False,
 'name': 'TJ Holowaychuk',
 'company': 'Apex',
 'blog': 'https://apex.sh',
 'location': 'Lond

In [ ]:
import time

users_data = []
i=0
for user in all_users:
    try:
        # Fetch user details
        response = requests.get(user["url"], headers=headers)
        response.raise_for_status()  # Raise an error if the request failed
        user_data = response.json()

        # Only proceed if 'login' key is present to confirm valid user data
        if "login" in user_data:
            user_info = {
                "login": user_data.get("login", ""),
                "name": user_data.get("name", ""),
                "company": user_data.get("company", "").strip().lstrip('@').upper() if user_data.get("company") else "",
                "location": user_data.get("location", ""),
                "email": user_data.get("email", ""),
                "hireable": user_data.get("hireable", ""),
                "bio": user_data.get("bio", ""),
                "public_repos": user_data.get("public_repos", 0),
                "followers": user_data.get("followers", 0),
                "following": user_data.get("following", 0),
                "created_at": user_data.get("created_at", ""),
            }
            users_data.append(user_info)
            print(i,end=" ")
            i+=1
        else:
            print(f"Skipping user {user.get('login', 'unknown')} due to missing data.")

    except requests.exceptions.RequestException as e:
        print(f"Request failed for user {user['login']}: {e}")
        continue  # Skip to the next user on failure

    time.sleep(1)  # Add a 1-second delay between requests

print(f"Total users processed: {len(users_data)}")


In [ ]:
len(users_data)

325

In [ ]:
df=pd.DataFrame(users_data)

In [ ]:
df.to_csv('users.csv', index=False)

In [ ]:
import requests
import time

repos_data = []
# Loop through each user and fetch their repositories
for user in all_users:
    try:
        user_repo_data = []  # List to accumulate repositories for this user
        page = 1  # Start with the first page for each user

        # Loop through pages to fetch up to 500 repositories
        while len(user_repo_data) < 500:
            # Fetch repositories from the current page
            response = requests.get(
                user["repos_url"],
                headers=headers,
                params={"per_page": 100, "page": page}
            )
            response.raise_for_status()
            page_repos = response.json()

            # Break if there are no more repositories
            if not page_repos:
                break

            user_repo_data.extend(page_repos)  # Add this page's repos to the user's list
            page += 1  # Move to the next page

            # Stop if we've accumulated 500 repositories
            if len(user_repo_data) >= 500:
                user_repo_data = user_repo_data[:500]  # Trim to exactly 500

            time.sleep(1)  # Delay to avoid hitting rate limits

        # Process each repository and add to repos_data
        for repo in user_repo_data:
            user_repo_info = {
                "login": user.get("login", ""),
                "full_name": repo.get("full_name", ""),
                "created_at": repo.get("created_at", ""),
                "stargazers_count": repo.get("stargazers_count", 0),
                "watchers_count": repo.get("watchers_count", 0),
                "language": repo.get("language", ""),
                "has_projects": repo.get("has_projects", False),
                "has_wiki": repo.get("has_wiki", False),
                "license": repo.get("license", {}).get("key", "") if repo.get("license") else ""  # Safely handle nested license key
            }
            repos_data.append(user_repo_info)

        print(f"Fetched {len(user_repo_data)} repositories for user {user['login']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed for user {user['login']}: {e}")
        continue  # Skip to the next user on failure

print("Repository data collection complete!")


Fetched 296 repositories for user tj
Fetched 46 repositories for user kunal-kushwaha
Fetched 41 repositories for user angelabauer
Fetched 169 repositories for user jlord
Fetched 49 repositories for user Elfocrash
Fetched 15 repositories for user alyssaxuu
Fetched 94 repositories for user eddiejaoude
Fetched 101 repositories for user daneden
Fetched 362 repositories for user LaravelDaily
Fetched 73 repositories for user jskeet
Fetched 500 repositories for user mattdesl
Fetched 475 repositories for user cloudflare
Fetched 97 repositories for user nicklockwood
Fetched 94 repositories for user jgthms
Fetched 125 repositories for user bizz84
Fetched 262 repositories for user samuelcolvin
Fetched 116 repositories for user Lissy93
Fetched 34 repositories for user jgilfelt
Fetched 35 repositories for user sonnysangha
Fetched 11 repositories for user nickbutcher
Fetched 18 repositories for user florina-muntenescu
Fetched 183 repositories for user pydanny
Fetched 450 repositories for user SaraVi

In [ ]:
repo_df=pd.DataFrame(repos_data)
repo_df.to_csv('repos.csv', index=False)

In [ ]:
repo_df

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license
0,tj,tj/archive,2016-02-06T19:50:22Z,1,1,Go,True,True,
1,tj,tj/archivex,2017-06-18T16:28:56Z,1,1,Go,True,True,bsd-3-clause
2,tj,tj/assert,2017-02-16T21:05:38Z,37,37,Go,True,True,mit
3,tj,tj/assert-equal-jsx,2016-06-15T14:52:43Z,5,5,JavaScript,True,True,mit
4,tj,tj/asset,2011-04-04T02:43:12Z,122,122,JavaScript,True,True,
...,...,...,...,...,...,...,...,...,...
39315,dturner,dturner/Universal-G-Code-Sender,2021-02-24T11:30:38Z,0,0,None,True,True,gpl-3.0
39316,dturner,dturner/vimconfig,2014-01-10T19:06:05Z,0,0,VimL,True,True,
39317,dturner,dturner/voltage-divider-calc,2020-04-21T20:00:59Z,0,0,None,True,True,
39318,dturner,dturner/walt,2016-06-15T09:39:45Z,0,0,Java,True,False,apache-2.0


In [ ]:
df = pd.read_csv("repositories.csv")

# Replace True/False with true/false
df = df.replace({True: "true", False: "false"})

# Save the modified DataFrame back to the original file
df.to_csv("repositories.csv", index=False)

print("CSV file updated successfully!")

CSV file updated successfully!


In [ ]:
df=pd.read_csv('repositories.csv')
df = df.rename(columns={"license": "license_name"})
df.to_csv('repositories.csv', index=False)

In [ ]:
user_df=pd.read_csv('users.csv')
repo_df=pd.read_csv('repositories.csv')

Q1

In [ ]:
user_df.sort_values(by='followers', ascending=False).head(5)

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,tj,TJ Holowaychuk,APEX,"London, UK",tj@apex.sh,NaN,NaN,296,50861,45,2008-09-18T22:37:28Z
1,kunal-kushwaha,Kunal Kushwaha,CIVO,"London, UK",contact@techwithkunal.com,NaN,Field CTO @civo. Founder @WeMakeDevs,46,30828,0,2018-08-25T15:01:59Z
2,angelabauer,Angela Yu,APPBREWERY,"London, UK",NaN,NaN,Medical doctor. Developer. Founder of the App ...,41,18641,0,2014-09-16T21:03:28Z
3,jlord,Jessica Lord,NaN,"London, England",NaN,NaN,Urban designer turned open source developer. S...,169,16356,30,2012-01-05T03:24:52Z
4,Elfocrash,Nick Chapsas,NaN,"London, UK",nick@nickchapsas.com,NaN,Content creator for C# & .NET,49,10582,0,2014-07-18T09:32:23Z


Q2

In [ ]:
user_df.sort_values(by='created_at').head(5)

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
189,aslakhellesoy,Aslak Hellesøy,SMARTBEAR,"London, UK",NaN,True,🥒,198,784,1,2008-02-26T22:58:16Z
308,tomstuart,Tom Stuart,NaN,London,hi@tomstu.art,NaN,NaN,59,524,4,2008-03-17T09:15:49Z
147,sole,sole,NaN,"London, UK",NaN,NaN,Biodegradable,242,979,12,2008-04-07T16:27:00Z
297,javache,Pieter De Baets,FACEBOOK,"London, UK",NaN,NaN,NaN,24,535,46,2008-04-07T20:41:02Z
236,philhawksworth,Phil Hawksworth,DENOLAND,London,phil@deno.com,NaN,Head of Developer Relations at @denoland \r\n\...,423,670,25,2008-04-08T16:52:05Z


Q3

In [ ]:
repo_df["license_name"].value_counts().head(3)

license_name
mit           10535
apache-2.0     4245
other          3022
Name: count, dtype: int64

Q4

In [ ]:
user_df["company"].value_counts().head(1)

company
FACEBOOK    14
Name: count, dtype: int64

Q5

In [ ]:
repo_df["language"].value_counts().head(1)

language
JavaScript    8646
Name: count, dtype: int64

Q6

In [ ]:
import numpy as np
condition=user_df["created_at"]>="2021-01-01"
users=np.array(user_df[condition]["login"])

In [ ]:
repo_df[repo_df["login"].isin(users)]["language"].value_counts().head(5)

language
Python        158
JavaScript     48
TypeScript     41
CSS            28
HTML           25
Name: count, dtype: int64

Q7

In [ ]:
repo_df.groupby("language")["stargazers_count"].mean().sort_values(ascending=False).head(5)

language
Sass                1157.333333
Rich Text Format     571.500000
Nushell              524.500000
Raku                 488.000000
ActionScript         484.125000
Name: stargazers_count, dtype: float64

Q8

In [ ]:
user_df["leader_strength"]=user_df["followers"]/(user_df["following"]+1)

In [ ]:
user_df.sort_values(by='leader_strength', ascending=False).head(5)

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,leader_strength
1,kunal-kushwaha,Kunal Kushwaha,CIVO,"London, UK",contact@techwithkunal.com,NaN,Field CTO @civo. Founder @WeMakeDevs,46,30828,0,2018-08-25T15:01:59Z,30828.0
2,angelabauer,Angela Yu,APPBREWERY,"London, UK",NaN,NaN,Medical doctor. Developer. Founder of the App ...,41,18641,0,2014-09-16T21:03:28Z,18641.0
4,Elfocrash,Nick Chapsas,NaN,"London, UK",nick@nickchapsas.com,NaN,Content creator for C# & .NET,49,10582,0,2014-07-18T09:32:23Z,10582.0
8,LaravelDaily,Laravel Daily,LARAVEL DAILY,"Vilnius, Lithuania / London, UK",info@laraveldaily.com,True,Team of Laravel developers with experience sin...,362,7278,0,2015-10-27T11:27:30Z,7278.0
11,cloudflare,Cloudflare,NaN,"San Francisco, London, Austin, Lisbon, Singapore",NaN,NaN,NaN,475,6433,0,2010-06-25T01:24:59Z,6433.0


Q9

In [ ]:
corr=user_df["followers"].corr(user_df["public_repos"])
corr

np.float64(0.028457985581810435)

Q10

In [ ]:
# !pip install statsmodels

In [ ]:
import statsmodels.api as sm
# Fit the regression model

X=user_df["public_repos"]
y=user_df["followers"]
# Add a constant to the model (for the intercept)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Print the regression summary
print(model.summary())
additional_followers_per_repo = model.params['public_repos']
print(f"Estimated additional followers per additional public repository: {additional_followers_per_repo:.2f}")

                            OLS Regression Results                            
Dep. Variable:              followers   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.2618
Date:                Tue, 29 Oct 2024   Prob (F-statistic):              0.609
Time:                        16:34:14   Log-Likelihood:                -3132.1
No. Observations:                 325   AIC:                             6268.
Df Residuals:                     323   BIC:                             6276.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1775.5664    208.452      8.518   

Q11

In [2]:
import pandas as pd
repo_df=pd.read_csv('repositories.csv')
user_df=pd.read_csv('users.csv')
# Convert has_projects and has_wiki to numeric (0 and 1)
repo_df['has_projects'] = repo_df['has_projects'].replace({"true": 1, "false": 0})
repo_df['has_wiki'] = repo_df['has_wiki'].replace({"true": 1, "false": 0})
corr=repo_df["has_wiki"].corr(repo_df["has_projects"])
corr

np.float64(0.4327496141171913)

Q12

In [ ]:
user_df[user_df["hireable"]!=True]

In [11]:
ans=user_df[user_df["hireable"]==True]["following"].mean()-user_df[user_df["hireable"]!=True]["following"].mean()
ans

np.float64(1067.538022631377)

Q13

In [20]:
user_df=pd.read_csv('users.csv')
user_df["bio"]

0                                                    NaN
1                  Field CTO @civo. Founder @WeMakeDevs 
2      Medical doctor. Developer. Founder of the App ...
3      Urban designer turned open source developer. S...
4                          Content creator for C# & .NET
                             ...                        
320    Software consultant. Author of spotify-tui. Lo...
321                                                  NaN
322                                                  </>
323    Hacker. Bot slayer. Ex-sysadmin. Bureaucramanc...
324    Android Developer Relations Engineer  \r\n@goo...
Name: bio, Length: 325, dtype: object

In [30]:
user_df.fillna("", inplace=True)
user_df["bio"]

0                                                       
1                  Field CTO @civo. Founder @WeMakeDevs 
2      Medical doctor. Developer. Founder of the App ...
3      Urban designer turned open source developer. S...
4                          Content creator for C# & .NET
                             ...                        
320    Software consultant. Author of spotify-tui. Lo...
321                                                     
322                                                  </>
323    Hacker. Bot slayer. Ex-sysadmin. Bureaucramanc...
324    Android Developer Relations Engineer  \r\n@goo...
Name: bio, Length: 325, dtype: object

In [26]:

# Load the CSV file
user_df = pd.read_csv("users.csv")

# Replace all NaN values with empty strings
user_df = user_df.fillna("")

# Replace all boolean values (True/False) with string "true"/"false"
user_df = user_df.replace({True: "true", False: "false"})

# Save the modified DataFrame back to the original CSV file
user_df.to_csv("users.csv", index=False)

In [29]:
user_df["bio"]

0                                                    NaN
1                  Field CTO @civo. Founder @WeMakeDevs 
2      Medical doctor. Developer. Founder of the App ...
3      Urban designer turned open source developer. S...
4                          Content creator for C# & .NET
                             ...                        
320    Software consultant. Author of spotify-tui. Lo...
321                                                  NaN
322                                                  </>
323    Hacker. Bot slayer. Ex-sysadmin. Bureaucramanc...
324    Android Developer Relations Engineer  \r\n@goo...
Name: bio, Length: 325, dtype: object

In [31]:
user_df=pd.read_csv('users.csv')
# Calculate bio length by counting words, handling missing values in "bio" with fillna
user_df["bio_length"] = user_df["bio"].fillna("").apply(lambda x: len(x.split()))

# Calculate correlation between bio length and followers
correlation = user_df["bio_length"].corr(user_df["followers"])

print("Correlation between bio length and followers:", correlation)


Correlation between bio length and followers: -0.02602401608912862


Q14

In [ ]:
# Convert 'created_at' to datetime
repo_df['created_at'] = pd.to_datetime(repo_df['created_at'], utc=True)

# Filter for repositories created on weekends (Saturday = 5, Sunday = 6)
weekend_repos = repo_df[repo_df['created_at'].dt.weekday >= 5]

# Count the number of weekend repositories per user
weekend_repo_counts = weekend_repos['login'].value_counts()

# Get the top 5 users with the most repositories created on weekends
weekend_repo_counts.head(5)

login
praveenscience    155
mattdesl          148
passy             146
valeman           145
CodeMaster7000    145
Name: count, dtype: int64

Q15

In [ ]:
import pandas as pd

# Load users data
users_df = pd.read_csv("users.csv")

# Calculate the fraction of hireable users with an email
hireable_with_email = users_df[(users_df['hireable'] == True) & (users_df['email'].notna())].shape[0]
hireable_total = users_df[users_df['hireable'] == True].shape[0]
fraction_hireable_with_email = hireable_with_email / hireable_total if hireable_total > 0 else 0

# Calculate the fraction of non-hireable users with an email
non_hireable_with_email = users_df[(users_df['hireable'].isna()) & (users_df['email'].notna())].shape[0]
non_hireable_total = users_df[users_df['hireable'].isna()].shape[0]
fraction_non_hireable_with_email = non_hireable_with_email / non_hireable_total if non_hireable_total > 0 else 0

# Calculate the difference
email_fraction_difference = fraction_hireable_with_email - fraction_non_hireable_with_email

print(f"Difference in email sharing: {email_fraction_difference}")
hireable_total
non_hireable_total
hireable_with_email


Difference in email sharing: 0.06703107019562715


46

Q16

In [ ]:
# Load the data
users_df = pd.read_csv("users.csv")

# Filter out missing names and extract the last word in each name as the surname
users_df['surname'] = users_df['name'].dropna().str.strip().str.split().str[-1]

# Count occurrences of each surname
surname_counts = users_df['surname'].value_counts()

# Find the most common surname count
max_count = surname_counts.max()

# Identify all surnames that match this maximum count, sorted alphabetically
most_common_surnames = surname_counts[surname_counts == max_count].index.sort_values().tolist()

# Display the most common surname(s) as a comma-separated string
result = ", ".join(most_common_surnames)
print(f"Most common surname(s): {result}")


Most common surname(s): Appleton, Brewery, Fuller, Greenfeld, Jackson, Li, Williams
